 obtain the image for the article. Start from 20mar2021. then raw->Ewald->ROI->mask->averaging->offset

In [39]:
# import self written libraries. These have to be in the same folder or the parent directory.
import sys, os
sys.path.append('./library/')
import reconstruct as rec
import fth_reconstruction as fth
import Phase_Retrieval as PhR
import pymaxp04 as maxi
import scipy.io as io
import cv2
from scipy import signal
from scipy.ndimage.filters import gaussian_filter
import random
import scipy.ndimage as ndimage
from cameras import load_greateyes, load_spe

import numpy as np
import matplotlib.pyplot as plt
from skimage.draw import circle
from skimage.draw import rectangle
from skimage.transform import resize
#interactive stuff
import ipywidgets as widgets
from IPython.display import display
import ipython_blocking
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.stats import pearsonr
import matplotlib.image as imsave
import scipy.constants as sc
from matplotlib.pyplot import figure, draw
from scipy import stats
from matplotlib import colors

from copy import deepcopy

In [40]:
%matplotlib widget

# Load images

Images will be loaded as .npy files, created by Stitching_**.ipynb

In [41]:
experimental_setup = {'ccd_dist': 18e-2, 'energy': 779.5, 'px_size' : 20e-6}

INSTRUMENT_KEYS = {
    'mono': 'scan/instrument/mono/energy',
    'undu_shift': 'scan/instrument/undulator/collection/shift',
    'ringcurrent': 'scan/instrument/petra_iii/current',
    'exitslit': 'scan/instrument/exit_slit/slit/y_gap'
    }

In [42]:
folder = '/home/mschndr/MaxP04_2103_commissioning/raw/'
fname = 'test_%05d.nxs'
p_list = [148]

In [45]:
pos= np.array((maxi.load_scanfile(folder + fname%p_list[0], INSTRUMENT_KEYS))["ccd"][0])
neg=pos.copy()

for p in p_list:
    print(fname%p)
    try:
        tmp = maxi.load_scanfile(folder + fname%p, INSTRUMENT_KEYS)
        if tmp.undu_shift > 0:
            pos +=np.array(tmp["ccd"][0])
            print('+')
        else:
            neg +=np.array(tmp["ccd"][0])
            print('-')
            
    except:
        print('Wait for file and retry.')

test_00148.nxs
-


In [46]:
pos2, neg2, intercept, slope=fth.load_both(pos,neg, auto_factor=True)
holo=pos2-neg2

neg=0.000 + 2.000*pos


# select energy of beam and pixel size of camera
this values will be part of the dictionary 'experimental_setup', containing:
- energy of the beam [eV]
- CCD camera px_size [m]
- CCD camera- sample distance [m]

In [47]:
import scipy.constants as cst
wl = cst.h * cst.c / (experimental_setup['energy'] * cst.e)
n_pixels_CCD=pos.shape[0]
px_size=experimental_setup["px_size"]
CCD_dist=experimental_setup['ccd_dist']
theta=1/2*np.arctan(0.5*px_size*n_pixels_CCD/CCD_dist)


lpx=0.5*wl/np.sin(2*theta)
print(lpx)

7.034865374634114e-09


# FIND CENTER
zoom on the center of the scattering pattern to select the center value

In [48]:
mi, ma = np.percentile(np.real(pos), (.01,99.99))

fig, ax = plt.subplots()
ax.imshow(np.real(pos), cmap = 'gray', vmin = mi, vmax = ma)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
x1, x2 = ax.get_xlim()
y2, y1 = ax.get_ylim()

center=np.array([ (y1+y2)/2,(x1+x2)/2 ])

In [119]:
center=np.array([1097.55411594,  945.08751243])

In [95]:
mask_pixel_raw=pos*0
mask_pixel=mask_pixel_raw.copy()

# generate mask for defective pixels (optional)

save the hologram, paint defective pixels in read and then reload the image

In [119]:
mi, ma = np.percentile(np.real(pos), (.01,99.99))

imsave.imsave("mask_pixel.png",(pos-np.amin(pos)), vmin=mi, vmax=ma, norm=color.LogNorm())

In [119]:
mask_pixel_raw=pos*0
mask_pixel_raw += imageio.imread('mask_pixel_2.png')[:,:,0]==255

mask_pixel_raw=np.minimum(mask_pixel_raw, np.ones(mask_pixel_raw.shape))

In [122]:
mask_pixel_raw=pos*0
mask_pixel_raw[:,1060:1110]=1
mask_pixel_raw[1140:1180,1000:]=1
mask_pixel_raw+=(pos<615)
mask_pixel_raw=np.minimum(mask_pixel_raw,1)

In [123]:
fig,ax=plt.subplots()
ax.imshow(pos*(1-mask_pixel_raw),vmax=2000, norm=colors.LogNorm())
fig,ax=plt.subplots()
ax.imshow(mask_pixel_raw)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# let's normalize them (12/03/21)

In [124]:
x=(pos[mask_pixel_raw==0]).flatten()
y=(neg[mask_pixel_raw==0]).flatten()

fig,ax=plt.subplots()
ax.scatter(x,y, marker='.',s=1)
ax.set_xlabel("pos")
ax.set_ylabel("neg")
res = stats.linregress(x,y)
values=np.array([np.amin(x), np.amax(x)])
ax.plot(values,  -res.intercept+res.slope*(values), 'b', label='fit')
ax.set_yscale("log")
ax.set_xscale("log")
ax.set_xlim(np.amin(x), np.amax(x))
ax.set_ylim(np.amin(y), np.amax(y))
print("neg=%0.3f + %0.3f*pos"%(res.intercept,res.slope))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

neg=0.000 + 1.000*pos


In [125]:
neg-=res.intercept
neg/=res.slope

In [52]:
pos2= pos.copy()
neg2= neg.copy()

# Ewald sphere projection (optional)
project the raw data (and the mask for defective pixels, if any) onto the Ewald sphere, knowing the camera distance, the center of the scattering pattern and the wavelenght used.

pos,neg,mask_pixel_raw ---> pos2,neg2,mask_pixel
The algorithm projects everything and then centers all the scattering patterns.

In [53]:
pos2,points2, points, values=PhR.inv_gnomonic_2(CCD=pos+1000,center=center, experimental_setup = experimental_setup)

center= [1157.11312815 1076.7163302 ] z= 0.18


In [54]:
neg2,points2, points, values=PhR.inv_gnomonic_2(CCD=neg+1000,center=center, experimental_setup = experimental_setup)

center= [1157.11312815 1076.7163302 ] z= 0.18


In [129]:
if np.amin(mask_pixel_raw)<1:
    mask_pixel,points2, points, values=PhR.inv_gnomonic_2(CCD=mask_pixel_raw,center=center, experimental_setup = experimental_setup, method="linear")
else:
    mask_pixel=mask_pixel_raw.copy()
mask_pixel[mask_pixel>0]=1
mask_pixel[mask_pixel<=0]=0
mask_pixel[pos2==0]=1

center= [1157.11312815 1076.7163302 ] z= 0.18


In [69]:
pos2bis= pos2-1000
neg2bis= neg2-1000
mask_pixelbis= mask_pixel.copy()

In [127]:
pos2= pos2bis.copy()
neg2= neg2bis.copy()
mask_pixel= mask_pixelbis.copy()

# Get rid of useless pixels at the edge after Ewald projection

In [131]:
fig,ax=plt.subplots()
ax.imshow(pos2*(1-mask_pixel),vmax=2000, norm=colors.LogNorm())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [132]:
yy,xx=np.where(mask_pixel==0)
start_y,end_y=np.amin(yy),np.amax(yy)
start_x,end_x=np.amin(xx),np.amax(xx)
print(start_y,end_y)
print(start_x,end_x)

0 2047
0 2047


In [133]:
side=np.maximum(end_y-start_y,end_x-start_x)
print(side)
center_crop=[(start_y+end_y)//2,(start_x+end_x)//2]

pos3=np.zeros((side,side))
neg3=np.zeros((side,side))
mask_pixel3=np.zeros((side,side))

pos3[0:end_y-start_y,0:end_x-start_x]=pos2[start_y:end_y,start_x:end_x]
neg3[0:end_y-start_y,0:end_x-start_x]=neg2[start_y:end_y,start_x:end_x]
mask_pixel3[0:end_y-start_y,0:end_x-start_x]=mask_pixel[start_y:end_y,start_x:end_x]

2047


In [134]:
side=np.minimum(end_y-start_y,end_x-start_x)
print(side)

pos3=np.zeros((side,side))
neg3=np.zeros((side,side))
mask_pixel3=np.zeros((side,side))

pos3[0:side,0:side]=pos2[start_y:start_y+side,start_x:start_x+side]
neg3[0:side,0:side]=neg2[start_y:start_y+side,start_x:start_x+side]
mask_pixel3[0:side,0:side]=mask_pixel[start_y:start_y+side,start_x:start_x+side]

2047


In [135]:
center_crop=[center[0]-start_y,center[1]-start_x]

In [136]:
pos3= fth.set_center(pos3, center_crop)
neg3= fth.set_center(neg3, center_crop)
mask_pixel3= fth.set_center(mask_pixel3, center_crop)

Shifted image by -53 pixels in x and -134 pixels in y.
Shifted image by -53 pixels in x and -134 pixels in y.
Shifted image by -53 pixels in x and -134 pixels in y.


In [137]:
crop=950
fig,ax=plt.subplots()
ax.imshow(pos3[crop:-crop,crop:-crop])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [138]:
pos2=pos3.copy()
neg2=neg3.copy()
mask_pixel=mask_pixel3.copy()

# ROI

You may select the reconstruction you want to look at. Do this by zooming into the image and putting in the x and y coordinates of the start and end points of your ROI.

Click on the button when you are finished to resume the execution of the code. 

In [139]:
abs_val=pos2+neg2

In [140]:
ROI=True
if ROI:
    fig, ax = fth.plot(np.abs(fth.reconstruct(abs_val)), colorbar = False, scale = (2,96))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [141]:
if ROI:
    x1, x2 = ax.get_xlim()
    y2, y1 = ax.get_ylim()
    side=np.maximum(x2-x1, y2-y1)/2
    roi_array = np.array([fth.integer((x1+x2)/2-side), fth.integer((x1+x2)/2+side), fth.integer((y1+y2)/2-side), fth.integer((y1+y2)/2+side)]) #xstart, xstop, ystart, ystop
    roi = np.s_[roi_array[2]:roi_array[3], roi_array[0]:roi_array[1]]

In [109]:
roi=(slice(1074, 1313, None), slice(1275, 1514, None))
roi=np.s_[1074:1313,1275:1514]

# Check the FTH reconstruction
use the sliders to find the right propagation distance, phase rotation or sub-pixel centering
You can select different modes of representation:

mode=
- \- : difference of the two images;
- \+ : sum of the two helicities, for topography image;
- / : ratio between the two images;
- -/+: ratio between difference and sum of the two helicity images

In [218]:
prop_dist_holo,phase_holo,dx_holo,dy_holo = 0,0,0,0
mode='+'
slider_prop,slider_phase, slider_dx, slider_dy = rec.focus(pos2, neg2, roi=np.s_[:,:], mask=1,prop_dist=prop_dist_holo,
                                                                           phase=phase_holo,dx=dx_holo,dy=dy_holo, experimental_setup=experimental_setup, operation=mode, max_prop_dist=2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='propagation[um]', layout=Layout(width='90%'), max=2.…

In [145]:
prop_dist_holo=slider_prop.value
phase_holo = slider_phase.value
dx_holo = slider_dx.value
dy_holo = slider_dy.value

In [149]:
from importlib import reload
reload(PhR)

<module 'Phase_Retrieval' from './library/Phase_Retrieval.py'>

# Select the support mask for the phase retrieval
Zoom on all RH and on the OH, everytime press on the button "Add RH". Then you will be able to individually threshold each aperture to create your support mask
The first RH would be the one on the center. Just zoom on another one to get the size precisely, the code will place that on the center automatically

In [155]:
RHdict={}

In [156]:
RHdict=PhR.holomask(holo=abs_val,plot_max=98.0, RHdict=RHdict)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Add RH', style=ButtonStyle())

Output()

# Build and visualize mask + artificial BS

set a maxium value with the sliders. Only regions included before where the image exceeds your value will be included in the support.
You can select maxim values for OH and for references separately.

In [157]:
RHdict["RH1"]["x"]=abs_val.shape[1]//2
RHdict["RH1"]["y"]=abs_val.shape[0]//2
RHdict

{'RH1': {'#': 1, 'r': 13, 'x': 1023, 'y': 1023},
 'RH2': {'#': 2, 'r': 16, 'x': 763, 'y': 163},
 'RH3': {'#': 3, 'r': 19, 'x': 305, 'y': 482},
 'RH4': {'#': 4, 'r': 15, 'x': 466, 'y': 1015},
 'RH5': {'#': 5, 'r': 15, 'x': 1207, 'y': 495},
 'RH6': {'#': 6, 'r': 138, 'x': 754, 'y': 638}}

In [158]:
mask_holes=np.zeros(abs_val.shape)

for key,i in RHdict.items():
    yy,xx=circle(i["y"], i["x"], i["r"])
    mask_holes[yy,xx]=i["#"]

In [161]:
reco=np.abs(fth.reconstruct(abs_val))
fig, ax = plt.subplots()
def p(x,y,z,k,w,l):
    
    hole_array=[x,y,z,k,w,l]
    
    toplot=np.zeros(reco.shape)
    for i in range(0,len(RHdict.keys())):
        toplot+=(reco*(mask_holes==(i+1))>hole_array[i])

    ax.imshow(toplot,cmap='binary')
    

    return

layout = widgets.Layout(width='90%')
style = {'description_width': 'initial'}

max_slider=350

slider_max_OH = widgets.FloatSlider(min = 0 , max = max_slider, step=0.0001, value= 0 , layout=layout,
                                   description = 'vmax_OH', style=style)
slider_max_ref = widgets.FloatSlider(min = 0 , max = max_slider, step=0.001, value= 20 , layout=layout,
                                   description = 'vmax_ref', style=style)
slider_max_ref2 = widgets.FloatSlider(min = 0 , max =max_slider, step=0.001, value= 3 , layout=layout,
                                   description = 'vmax_ref2', style=style)
slider_max_ref3 = widgets.FloatSlider(min = 0 , max = max_slider, step=0.001, value= 0.6 , layout=layout,
                                   description = 'vmax_ref3', style=style)
slider_max_ref4 = widgets.FloatSlider(min = 0 , max = max_slider, step=0.001, value= 0.9 , layout=layout,
                                   description = 'vmax_ref4', style=style)
slider_max_ref5 = widgets.FloatSlider(min = 0 , max = max_slider, step=0.001, value= 0.9 , layout=layout,
                                   description = 'vmax_ref5', style=style)

widgets.interact(p, x=slider_max_OH,y=slider_max_ref, z=slider_max_ref2, k=slider_max_ref3, w=slider_max_ref4, l=slider_max_ref5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='vmax_OH', layout=Layout(width='90%'), max=350.0, ste…

<function __main__.p(x, y, z, k, w, l)>

In [162]:
vmax=[slider_max_OH.value,slider_max_ref.value,slider_max_ref2.value,slider_max_ref3.value,slider_max_ref4.value,slider_max_ref5.value]

In [163]:
reco=np.abs(fth.reconstruct(abs_val))
supportmask=np.zeros(reco.shape)
for i in range(0,len(RHdict.keys())):
    supportmask+=reco*(mask_holes==(i+1))>vmax[i]
supportmask= np.minimum(supportmask, np.ones((supportmask.shape[0],supportmask.shape[1])))

In [164]:
yy,xx=np.where(supportmask==1)
ymin,ymax=np.amin(yy), np.amax(yy)
xmin,xmax=np.amin(xx), np.amax(xx)

fig,ax=plt.subplots( figsize=(5,5))
ax.set_title("your support mask")
ax.imshow(supportmask[ymin-10:ymax+10,xmin-10:xmax+10], cmap="binary")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [165]:
np.save("supportmask.npy",supportmask)

In [86]:
supportmask=np.load("supportmask.npy")

# Normalize pos2 and neg2

In [166]:
negative_roi=pos2*0+1*supportmask
negative_roi[roi]=0

In [167]:
x=(np.abs(fth.reconstruct(pos2))[negative_roi==1]).flatten()
y=(np.abs(fth.reconstruct(neg2))[negative_roi==1]).flatten()

In [168]:
fig,ax=plt.subplots()
ax.scatter(x,y, marker='.',s=1)
ax.set_xlabel("pos2")
ax.set_ylabel("neg2")
res = stats.linregress(x,y)
values=np.array([np.amin(x), np.amax(x)])
ax.plot(values,   res.slope*(values)+res.intercept, 'b', label='fit')
#ax.set_yscale("log")
#ax.set_xscale("log")
ax.set_xlim(np.amin(x), np.amax(x))
ax.set_ylim(np.amin(y), np.amax(y))
print("neg2=",res.intercept,"+",res.slope,"*pos")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

neg2= 0.0 + 1.0 *pos


In [169]:
neg2-=res.intercept
neg2/=res.slope

# take out offset
first select a range of values, then find a value to be subtracted to the scattering patterns so that there is no constant background and the lower values of the outer fringes of the hologram is 0
The code will subtract the offset. In case of pixels with negative values, they will later be set to 0 for phase retrieval (as the square root of intensity is needed

In [170]:
offset={'+':0,'-':0}

positive helicity first

In [171]:
from matplotlib.gridspec import GridSpec
holo_pos=pos2.copy()
#Scan on line
line=holo_pos.shape[0]//2
cropping=0
from_to=[cropping,holo_pos.shape[0]-cropping]
dir='h'

lineavg=1
linewidth=np.maximum(lineavg,2)
recon=holo_pos

line_mark=np.zeros(recon.shape)
if dir=='h':
    linescan=recon[line-lineavg//2:line+lineavg//2+1,from_to[0]:from_to[1]]
    linescan=np.sum(linescan, axis=0)
    line_mark[line-linewidth:line+linewidth,from_to[0]:from_to[1]]=1
    index=1
    linescan_reduced=recon[line,from_to[0]:from_to[1]]
elif dir=='v':
    linescan=recon[from_to[0]:from_to[1],line-lineavg//2:line+lineavg//2+1]
    linescan=np.sum(linescan, axis=1)
    line_mark[from_to[0]:from_to[1],line-linewidth:line+linewidth]=1
    index=0
    linescan_reduced=recon[from_to[0]:from_to[1],line]
line_mark=np.ma.masked_where(line_mark==0, line_mark)

f = plt.figure()
gs=GridSpec(1,2, left=0, right=0.98,top=0.77, bottom=0.23)
ax = f.add_subplot(gs[0,0])
ax2 = f.add_subplot(gs[0,1])
ax.set_title('Far field image - + hel.')
mi, ma = np.percentile(recon, (0,90))
ax.imshow(recon[from_to[0]:from_to[1],from_to[0]:from_to[1]],vmin=mi,vmax=ma,cmap='coolwarm')
#ax.imshow(line_mark[from_to[0]:from_to[1],from_to[0]:from_to[1]],vmin=0,vmax=1,cmap='binary')
#ax.imshow(satmap[from_to[0]:from_to[1],from_to[0]:from_to[1]], vmin = -1+0.01, vmax = 8+0.01,cmap='Set1',alpha=0.4)

title='linescan - select yrange here'
ax2.set_title(title)
ax2.set_xlabel('pixels')
ax2.set_ylabel('electrons')
#ax2.plot(np.linspace(from_to[0], from_to[1],from_to[1]-from_to[0]), np.log10(linescan), '-')
ax2.set_yscale('log')
#ax2.set_ylim(0,10000000)
ax2.plot(np.linspace(from_to[0], from_to[1],from_to[1]-from_to[0]), (linescan), '-')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [172]:
ymin, ymax = ax2.get_ylim()

In [173]:
fig, ax = plt.subplots()
def p(x):
    toplot=holo_pos<x
    toplot=np.ones(holo_pos.shape)*toplot
    ax.imshow(toplot, cmap="binary")

    return

layout = widgets.Layout(width='90%')
style = {'description_width': 'initial'}

slider_offset = widgets.FloatSlider(min = fth.integer(ymin)-770 , max = fth.integer(ymax), step=1, value= offset['+'] , layout=layout, description = 'offset', style=style)

widgets.interact(p, x=slider_offset)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='offset', layout=Layout(width='90%'), max=1414.0, min…

<function __main__.p(x)>

In [174]:
offset['+']=slider_offset.value

negative helcity second

In [175]:
holo_pos=neg2.copy()
#Scan on line
line=holo_pos.shape[0]//2
cropping=0
from_to=[cropping,holo_pos.shape[0]-cropping]
dir='h'

lineavg=1
linewidth=np.maximum(lineavg,2)
recon=holo_pos

line_mark=np.zeros(recon.shape)
if dir=='h':
    linescan=recon[line-lineavg//2:line+lineavg//2+1,from_to[0]:from_to[1]]
    linescan=np.sum(linescan, axis=0)
    line_mark[line-linewidth:line+linewidth,from_to[0]:from_to[1]]=1
    index=1
    linescan_reduced=recon[line,from_to[0]:from_to[1]]
elif dir=='v':
    linescan=recon[from_to[0]:from_to[1],line-lineavg//2:line+lineavg//2+1]
    linescan=np.sum(linescan, axis=1)
    line_mark[from_to[0]:from_to[1],line-linewidth:line+linewidth]=1
    index=0
    linescan_reduced=recon[from_to[0]:from_to[1],line]
line_mark=np.ma.masked_where(line_mark==0, line_mark)

f = plt.figure()
gs=GridSpec(1,2, left=0, right=0.98,top=0.77, bottom=0.23)
ax = f.add_subplot(gs[0,0])
ax2 = f.add_subplot(gs[0,1])
ax.set_title('Far field image - + hel.')
mi, ma = np.percentile(recon, (0,90))
ax.imshow(recon[from_to[0]:from_to[1],from_to[0]:from_to[1]],vmin=mi,vmax=ma,cmap='coolwarm')
#ax.imshow(line_mark[from_to[0]:from_to[1],from_to[0]:from_to[1]],vmin=0,vmax=1,cmap='binary')
#ax.imshow(satmap[from_to[0]:from_to[1],from_to[0]:from_to[1]], vmin = -1+0.01, vmax = 8+0.01,cmap='Set1',alpha=0.4)

title='linescan - select yrange here'
ax2.set_title(title)
ax2.set_xlabel('pixels')
ax2.set_ylabel('electrons')
#ax2.plot(np.linspace(from_to[0], from_to[1],from_to[1]-from_to[0]), np.log10(linescan), '-')
ax2.set_yscale('log')
#ax2.set_ylim(0,10000000)
ax2.plot(np.linspace(from_to[0], from_to[1],from_to[1]-from_to[0]), (linescan), '-')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [119]:
ymin, ymax = ax2.get_ylim()

In [176]:
fig, ax = plt.subplots()
def p(x):
    toplot=holo_pos<x
    toplot=np.ones(holo_pos.shape)*toplot
    ax.imshow(toplot, cmap="binary")

    return

layout = widgets.Layout(width='90%')
style = {'description_width': 'initial'}

slider_offset = widgets.FloatSlider(min = fth.integer(ymin)-50 , max = fth.integer(ymax), step=1, value= offset['-'] , layout=layout, description = 'offset', style=style)

widgets.interact(p, x=slider_offset)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=454.0, description='offset', layout=Layout(width='90%'), max=1414.0, m…

<function __main__.p(x)>

In [177]:
offset['-']=slider_offset.value

In [178]:
pos2=pos2-offset['+']
neg2=neg2-offset['-']

In [182]:
fig,ax=plt.subplots(1,2, figsize=(10,5))
ax[0].plot(pos2[:,pos.shape[0]//2])
ax[0].plot(neg2[:,pos.shape[0]//2])
#ax[0].set_ylim(100,160000)
ax[0].set_yscale("log")
ax[1].plot(np.abs(pos2-neg2)[:,pos.shape[0]//2])
#ax[1].set_ylim(-100000,100000)
ax[1].set_yscale("log")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [289]:
pos_cropped,neg_cropped, mask_pixel_cropped, roi_cropped_array, supportmask_cropped=pos2.copy(), neg2.copy(), mask_pixel.copy(), roi_array.copy(), supportmask.copy()
roi_cropped=np.s_[roi_cropped_array[2]:roi_cropped_array[3], roi_cropped_array[0]: roi_cropped_array[1]]

# crop if needed

In [263]:
crop=700
pos_cropped,neg_cropped, mask_pixel_cropped=pos2[crop:-crop,crop:-crop],neg2[crop:-crop,crop:-crop], mask_pixel[crop:-crop,crop:-crop]

In [280]:
roi_array_cropped=roi_array*(pos_cropped.shape[0]/pos2.shape[0])
roi_array_cropped=roi_array_cropped.astype(int)
roi_cropped = np.s_[roi_array_cropped[2]:roi_array_cropped[3], roi_array_cropped[0]:roi_array_cropped[1]]

In [281]:
roi_array_cropped

array([193, 284, 156, 247])

array([193.43624817, 284.14899853, 156.45578896, 247.16853933])

In [272]:
thr=0.5
supportmask_cropped=fth.reconstruct(fth.reconstructCDI(supportmask)[crop:-crop,crop:-crop])
supportmask_cropped=1*(np.abs(supportmask_cropped)>1)

In [273]:
fig,ax=plt.subplots(2,2)
ax[0,0].imshow(pos2, norm=colors.LogNorm())
ax[0,1].imshow(supportmask, vmin=0, vmax=1)
ax[1,0].imshow(pos_cropped, norm=colors.LogNorm())
ax[1,1].imshow(supportmask_cropped, vmin=0, vmax=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# FULLY COHERENT PHASE RETRIEVAL
Now you will retrieve the phase from the scattering pattern collected.
First, if necessary and if it's not already included in mask_pixel, select the diameter of the beamstop. The pixels inside this diameter will be masked and the magnitude constraint will not be applied to them.
bsmask_p and bsmask_n are the set of points with incorrect values (covered by BS, hot and cold pixels, pixels with negative values after offset subtraction). They will be masked during phase retrieval and the magnitude constraint will not be imposed on them.

In [290]:
bs_diam=0
BS2=[pos_cropped.shape[0]//2,pos_cropped.shape[1]//2,bs_diam//2]
yy,xx = circle(pos_cropped.shape[0]//2, pos_cropped.shape[1]//2, bs_diam//2)

real_object=False

In [291]:
bsmask_p=mask_pixel_cropped.copy()
bsmask_p[pos_cropped<=0]=1
bsmask_p[yy,xx]=1
bsmask_n=mask_pixel_cropped.copy()
bsmask_n[neg_cropped<=0]=1
bsmask_n[yy,xx]=1

### select a starting guess for your phase retrieval algorithm

Startimage = 0 will make you start with a random phase.
Every other complex array as an input will be used as the starting phase for the algorithm. Popular choices are, for example, the fourier transform of the FTH reconstruction of pos2 multiplied by the support mask, or just the support mask multiplied by a factor.

In [292]:
prop_dist,phase,dx,dy=0,0,0,0
prop_dist_pc,phase_pc,dx_pc,dy_pc=0,0,0,0

In [293]:
reload(rec)
#this one
#CDI
plt.rcParams['figure.dpi'] = 100
print("CDI - larger mask")
Startimage=supportmask_cropped*(pos_cropped*0+1.5*1e7*np.exp(-1j*0.7))
Startimage=np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(Startimage)))


R_apod=None
Startgamma=np.ones(pos_cropped.shape)
#Startgamma[pos_cropped.shape[0]//2, pos_cropped.shape[1]//2]=0.7




#retrieve pos image

for i in range(2):
    
    print("############ %i   -   CDI"%i)
    
    retrieved_p,Error_diff_p,Error_supp=PhR.PhaseRtrv_GPU(diffract=np.sqrt(np.maximum(pos_cropped,np.zeros(pos_cropped.shape))), mask=supportmask_cropped, mode='RAAR',
                 beta_zero=0.5, Nit=700, beta_mode='arctan', plot_every=349,Phase=Startimage,seed=False,real_object=real_object, BS=BS2, bsmask=bsmask_p,average_img=30, Fourier_last=True)
    retrieved_p,Error_diff_p2,Error_supp=PhR.PhaseRtrv_GPU(diffract=np.sqrt(np.maximum(pos_cropped,np.zeros(pos_cropped.shape))), mask=supportmask_cropped, mode='ER',
                 beta_zero=0.5, Nit=50, beta_mode='const', plot_every=24,
                 Phase=retrieved_p,seed=False,real_object=real_object, BS=BS2, bsmask=bsmask_p,average_img=30, Fourier_last=True)
    retrieved_n,Error_diff_n2,Error_supp=PhR.PhaseRtrv_GPU(diffract=np.sqrt(np.maximum(neg_cropped,np.zeros(neg_cropped.shape))), mask=supportmask_cropped, mode='ER',
                 beta_zero=0.5, Nit=50, beta_mode='const', plot_every=24,
                 Phase=retrieved_p,seed=False,real_object=real_object, BS=BS2, bsmask=bsmask_n,average_img=30, Fourier_last=True)
    mode="-"
    slider_prop,slider_phase, slider_dx, slider_dy = rec.focusCDI(retrieved_p, retrieved_n, roi=roi_cropped, mask=supportmask_cropped, phase=phase, prop_dist=prop_dist,
                                                                               experimental_setup=experimental_setup, operation=mode, max_prop_dist=0.4)
    mode="+"
    slider_prop,slider_phase, slider_dx, slider_dy = rec.focusCDI(retrieved_p, retrieved_n, roi=roi_cropped, mask=supportmask_cropped, phase=phase, prop_dist=prop_dist,
                                                                               experimental_setup=experimental_setup, operation=mode, max_prop_dist=0.4)
    #CDI_PC
    print("############ %i   -   CDI_pc"%i)
    pos3=(np.abs(retrieved_p)**2)*bsmask_p+ np.maximum(pos_cropped,np.zeros(pos_cropped.shape))*(1-bsmask_p)
    neg3=(np.abs(retrieved_n)**2)*bsmask_n+ np.maximum(neg_cropped,np.zeros(neg_cropped.shape))*(1-bsmask_n)
    Startimage =  retrieved_p.copy()

    #retrieve pos image
    retrieved_p_pc,Error_diff_p_pc,Error_supp, gamma_p=PhR.PhaseRtrv_with_RL(diffract=np.sqrt(pos3), mask=supportmask_cropped, mode='RAAR',
                 beta_zero=0.5, Nit=700, beta_mode="arctan", gamma=Startgamma, RL_freq=30, RL_it=30,
                 plot_every=349,Phase=Startimage, seed=False, real_object=False, BS=BS2, bsmask=np.zeros(bsmask_p.shape), average_img=30, Fourier_last=True, R_apod=R_apod)

    retrieved_p_pc,Error_diff_p_pc2,Error_supp, gamma_p=PhR.PhaseRtrv_with_RL(diffract=np.sqrt(pos3), mask=supportmask_cropped, mode='ER',
                 beta_zero=0.5, Nit=50, beta_mode='const', gamma=gamma_p, RL_freq=30, RL_it=30,
                plot_every=24,Phase=retrieved_p_pc,real_object=False, BS=BS2, bsmask=np.zeros(bsmask_p.shape), average_img=30, Fourier_last=True, R_apod=R_apod)
    retrieved_n_pc,Error_diff_n_pc2,Error_supp, gamma_n=PhR.PhaseRtrv_with_RL(diffract=np.sqrt(neg3), mask=supportmask_cropped, mode='ER',
                 beta_zero=0.5, Nit=50, beta_mode='const', gamma=gamma_p, RL_freq=30, RL_it=30,
                plot_every=24,Phase=retrieved_p_pc,real_object=False, BS=BS2, bsmask=np.zeros(bsmask_n.shape), average_img=30, Fourier_last=True, R_apod=R_apod)

    mode="-"
    slider_prop,slider_phase, slider_dx, slider_dy = rec.focusCDI(retrieved_p_pc, retrieved_n_pc, roi_cropped, mask=supportmask_cropped,prop_dist=prop_dist_pc,
                                                                               phase=phase_pc,dx=dx_pc,dy=dy_pc, experimental_setup=experimental_setup, operation=mode, max_prop_dist=0.7)
    mode="+"
    slider_prop,slider_phase, slider_dx, slider_dy = rec.focusCDI(retrieved_p_pc, retrieved_n_pc, roi_cropped, mask=supportmask_cropped,prop_dist=prop_dist_pc,
                                                                               phase=phase_pc,dx=dx_pc,dy=dy_pc, experimental_setup=experimental_setup, operation=mode, max_prop_dist=0.7)
    #retrieved_p=retrieved_p_pc.copy()
    Startimage =  retrieved_p_pc.copy()
    Startgamma =  gamma_p.copy()

CDI - larger mask
############ 0   -   CDI
using phase given
mode= RAAR     beta_mode= arctan
# 0    beta= 0.9354686820426836    Error_diffr= 8.786526843385428    Error_supp= -40.95432299282269
# 349    beta= 0.7414550869157752    Error_diffr= -7.59643608807965    Error_supp= -43.707161528992756
# 698    beta= 0.544772880989826    Error_diffr= -13.583604821931388    Error_supp= -43.60265525969375
using phase given
mode= ER     beta_mode= const
# 0    beta= 0.5    Error_diffr= -15.123692328088133    Error_supp= -43.59850462251256
# 24    beta= 0.5    Error_diffr= -15.129305955537264    Error_supp= -43.59731667733235
# 48    beta= 0.5    Error_diffr= -15.132977652271771    Error_supp= -43.596215752347625
using phase given
mode= ER     beta_mode= const
# 0    beta= 0.5    Error_diffr= -15.000190643340295    Error_supp= -43.64353351320655
# 24    beta= 0.5    Error_diffr= -15.009348420285296    Error_supp= -43.67349665700502
# 48    beta= 0.5    Error_diffr= -15.01278897909048    Error_sup

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='propagation[um]', layout=Layout(width='90%'), max=0.…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='propagation[um]', layout=Layout(width='90%'), max=0.…

...

############ 1   -   CDI_pc
using phase given
mode= RAAR     beta_mode= arctan
# 0    beta= 0.9354686820426836    Error_diffr= (-20.789982899436893-0.001780855391155993j)    Error_supp= -43.80237779972966
# 349    beta= 0.7414550869157752    Error_diffr= (-24.511808659622737+0.036707592927731486j)    Error_supp= -43.75036593217915
# 698    beta= 0.544772880989826    Error_diffr= (-35.95029018068459-0.05004833598010158j)    Error_supp= -44.29177292387575
using phase given
mode= ER     beta_mode= const
# 0    beta= 0.5    Error_diffr= (-37.44903161217391+0.05263362692306166j)    Error_supp= -44.52443583460561
# 24    beta= 0.5    Error_diffr= (-38.86467417011817-0.06768301684055152j)    Error_supp= -44.35739030518831
# 48    beta= 0.5    Error_diffr= (-38.81552724085014-0.06709547379902484j)    Error_supp= -44.34179419828324
using phase given
mode= ER     beta_mode= const
# 0    beta= 0.5    Error_diffr= (-39.86495197198808+0.07063223859872242j)    Error_supp= -44.47502710936354
# 24    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='propagation[um]', layout=Layout(width='90%'), max=0.…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='propagation[um]', layout=Layout(width='90%'), max=0.…

# check your results

In [294]:
print(experimental_setup)
mode="+"
slider_prop,slider_phase, slider_dx, slider_dy = rec.focusCDI(retrieved_p, retrieved_n, roi=np.s_[:,:], mask=supportmask_cropped, prop_dist=prop_dist,
                                                                           phase=phase,dx=dx,dy=dy, experimental_setup=experimental_setup, operation=mode, max_prop_dist=0.7)

{'ccd_dist': 0.18, 'energy': 779.5, 'px_size': 2e-05}


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='propagation[um]', layout=Layout(width='90%'), max=0.…

In [286]:
print(experimental_setup)
mode="+"
slider_prop,slider_phase, slider_dx, slider_dy = rec.focusCDI(retrieved_p, retrieved_n, roi=roi_cropped, mask=supportmask_cropped, prop_dist=prop_dist,
                                                                           phase=phase,dx=dx,dy=dy, experimental_setup=experimental_setup, operation=mode, max_prop_dist=0.7)

{'ccd_dist': 0.18, 'energy': 779.5, 'px_size': 2e-05}


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='propagation[um]', layout=Layout(width='90%'), max=0.…

In [295]:
print(experimental_setup)
mode="+"
slider_prop,slider_phase, slider_dx, slider_dy = rec.focusCDI(retrieved_p_pc, retrieved_n_pc, roi=np.s_[:,:], mask=supportmask_cropped, prop_dist=prop_dist_pc,
                                                                           phase=phase_pc,dx=dx_pc,dy=dy_pc, experimental_setup=experimental_setup, operation=mode, max_prop_dist=17)

{'ccd_dist': 0.18, 'energy': 779.5, 'px_size': 2e-05}


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='propagation[um]', layout=Layout(width='90%'), max=17…

In [287]:
print(experimental_setup)
mode="-"
slider_prop,slider_phase, slider_dx, slider_dy = rec.focusCDI(retrieved_p_pc, retrieved_n_pc, roi_cropped, mask=supportmask_cropped, prop_dist=prop_dist_pc,
                                                                           phase=phase_pc,dx=dx_pc,dy=dy_pc, experimental_setup=experimental_setup, operation=mode, max_prop_dist=17)

{'ccd_dist': 0.18, 'energy': 779.5, 'px_size': 2e-05}


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='propagation[um]', layout=Layout(width='90%'), max=17…

In [211]:
prop_dist_pc=slider_prop.value
phase_pc = slider_phase.value
dx_pc = slider_dx.value
dy_pc = slider_dy.value

### check the retrieved MCF function

In [214]:
crop=2
factor=4

gamma=gamma_n.copy()
fig,ax = plt.subplots(1,3, figsize=(15.1,4.1))

MCF=np.abs(np.fft.fftshift(np.fft.fft2(np.fft.fftshift((gamma)))))[crop:-crop,crop:-crop]
extent = [-MCF.shape[0]//2*lpx*1e6 , MCF.shape[0]//2*lpx*1e6, -MCF.shape[1]//2*lpx*1e6 , MCF.shape[1]//2*lpx*1e6]

ax[0].set_aspect('equal')
m=ax[0].imshow(MCF, cmap="jet", extent=extent, norm=colors.LogNorm())
divider = make_axes_locatable(ax[0])
cax = divider.append_axes("right", size="5%", pad=0.05)
ax[0].set_title("Degree of coherence")
ax[0].set_xlabel("Separation $(\mu m)$")
ax[0].set_ylabel("Separation $(\mu m)$")

crop=gamma.shape[0]//2-4

MCF=np.abs(gamma)[crop:-crop,crop:-crop]
extent = [-MCF.shape[0]//2*1/lpx/1e6 , MCF.shape[0]//2*1/lpx/1e6, -MCF.shape[1]//2*1/lpx/1e6 , MCF.shape[1]//2*1/lpx/1e6]

ax[1].set_aspect('equal')
m2=ax[1].imshow(MCF, cmap="jet", extent=extent)
divider = make_axes_locatable(ax[1])
cax2 = divider.append_axes("right", size="5%", pad=0.05)
ax[1].set_title("Degree of coherence")
ax[1].set_xlabel("Separation $(\mu m)$")
ax[1].set_ylabel("Separation $(\mu m)$")

from scipy.interpolate import interp2d

x = np.linspace(0, MCF.shape[0], MCF.shape[0])
y = np.linspace(0, MCF.shape[1], MCF.shape[1])
X, Y = np.meshgrid(x, y)
Z = MCF.copy()#np.sin(np.pi*X/2) * np.exp(Y/2)

x2 = np.linspace(0, MCF.shape[0], MCF.shape[0]*factor)
y2 = np.linspace(0, MCF.shape[1], MCF.shape[1]*factor)
f = interp2d(x, y, Z, kind='cubic')
Z2 = f(x2, y2)


MCF=Z2.copy()
extent = [-MCF.shape[0]//2*1/lpx/1e6 , MCF.shape[0]//2*1/lpx/1e6, -MCF.shape[1]//2*1/lpx/1e6 , MCF.shape[1]//2*1/lpx/1e6]

ax[2].set_aspect('equal')
m2=ax[2].imshow(MCF, cmap="jet", extent=extent)
divider = make_axes_locatable(ax[2])
cax2 = divider.append_axes("right", size="5%", pad=0.05)
ax[2].set_title("Degree of coherence")
ax[2].set_xlabel("Separation $(\mu m)$")
ax[2].set_ylabel("Separation $(\mu m)$")

plt.colorbar(m, cax=cax)
plt.colorbar(m2, cax=cax2)
#ax.set_axis_off()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Save what you have done

In [627]:
data={}

In [758]:
data["experimental_setup"]=experimental_setup.copy()
data["center"]=center.copy()

In [759]:
data["px_size"]=lpx
data["center"]=center.copy()

In [760]:
data["raw"]={"+":pos.copy(), "-":neg.copy(), "bsmask":mask_pixel_raw.copy()}

In [761]:
data["holo"]={"+":pos2.copy(), "-":neg2.copy(), "bsmask":mask_pixel.copy(), "offset":offset.copy()}
data["holo"]["ROI"]=roi_array.copy()
data["holo"]["supportmask"]=supportmask.copy()

In [779]:
data["CDI"]={}
data["CDI_pc"]={}
data["CDI"]["ROI"]=roi_array_cropped.copy()
data["CDI_pc"]["ROI"]=roi_array_cropped.copy()

In [780]:
data["CDI"]={"+":retrieved_p.copy(), "-":retrieved_n.copy()}
data["CDI"]["holo"]={"+":pos_cropped.copy(), "-":neg_cropped.copy(), "bsmask":{"+":bsmask_p, "-":bsmask_n}, "offset":offset.copy()}

In [145]:
data["CDI_pc"]={"+":retrieved_p_pc.copy(), "-":retrieved_n_pc.copy()}
data["CDI_pc"]["holo"]={"+":pos3.copy(), "-":neg3.copy()}

In [146]:
data["CDI"]["supportmask"]=supportmask_cropped.copy()
data["CDI_pc"]["supportmask"]=supportmask_cropped.copy()

In [148]:
data["CDI_pc"]["gamma"]={"+":gamma_p.copy(), "-":gamma_n.copy()}

In [149]:
data["CDI_pc"]["focus"]={"prop_dist": prop_dist_pc, "phase": phase_pc,"dx": dx_pc, "dy": dy_pc}
data["CDI"]["focus"]={"prop_dist": prop_dist, "phase": phase,"dx": dx, "dy": dy}

In [150]:
filename1='./11009916/processed/CDI_new/s1909a_B2_2order/s1909a_B2_21gen2021'

In [151]:
filename='./11009916/processed/CDI_new/s1909a_B2_2order/s1909a_B2_'

In [788]:
folder_save='./Results/11008801/CDI/s1909a/'

In [152]:
PhR.create_hdf5(data,filename+"06apr2021")

dict_keys(['CDI', 'CDI_pc', 'ROI', 'ROI_RH', 'center', 'comment', 'experimental_setup', 'hazimuthal_integrals', 'holo', 'px_size', 'raw', 'stitching_details'])
create group CDI
---
CDI , <class 'dict'>
dict
dict_keys(['+', '-', 'focus', 'holo', 'supportmask', 'supportmask2'])
create group +
---
+ , <class 'numpy.ndarray'>
dataset created
---
create group -
---
- , <class 'numpy.ndarray'>
dataset created
---
create group focus
---
focus , <class 'dict'>
dict
dict_keys(['prop_dist', 'phase', 'dx', 'dy'])
create group prop_dist
---
prop_dist , <class 'numpy.float64'>
dataset created
---
create group phase
---
phase , <class 'numpy.float64'>
dataset created
---
create group dx
---
dx , <class 'numpy.float64'>
dataset created
---
create group dy
---
dy , <class 'numpy.float64'>
dataset created
---
---
create group holo
---
holo , <class 'dict'>
dict
dict_keys(['+', '-', 'bsmask', 'offset'])
create group +
---
+ , <class 'numpy.ndarray'>
dataset created
---
create group -
---
- , <class 'num

In [8]:
data=PhR.read_hdf5(filename+"18mar2021")

###  CDI
---
• +                    <class 'numpy.ndarray'>
• -                    <class 'numpy.ndarray'>
###  focus
---
• dx                    <class 'numpy.float64'>
• dy                    <class 'numpy.float64'>
• phase                    <class 'numpy.float64'>
• prop_dist                    <class 'numpy.float64'>
---
###  holo
---
• +                    <class 'numpy.ndarray'>
• -                    <class 'numpy.ndarray'>
###  bsmask
---
• +                    <class 'numpy.ndarray'>
• -                    <class 'numpy.ndarray'>
---
###  offset
---
• +                    <class 'numpy.float64'>
• -                    <class 'numpy.float64'>
---
---
• supportmask                    <class 'numpy.ndarray'>
• supportmask2                    <class 'numpy.ndarray'>
---
###  CDI_pc
---
• +                    <class 'numpy.ndarray'>
• -                    <class 'numpy.ndarray'>
###  focus
---
• dx                    <class 'numpy.float64'>
• dy                    <class 'numpy.fl

In [236]:
experimental_setup=data["experimental_setup"].copy()
center=data["center"].copy()
lpx=data["px_size"]
pos,neg,mask_pixel_raw=data["raw"]["+"],data["raw"]["-"],data["raw"]["bsmask"]
pos2,neg2,mask_pixel=data["holo"]["+"].copy(),data["holo"]["-"].copy(),data["holo"]["bsmask"].copy()
offset=data["holo"]["offset"].copy()
roi2=tuple(data["holo"]["ROI"])
roi=np.s_[roi2[0]:roi2[1],roi2[2]:roi2[3]]
retrieved_p,retrieved_n=data["CDI"]["+"].copy(),data["CDI"]["-"].copy()
bsmask_p,bsmask_n=data["CDI"]["holo"]["bsmask"]["+"].copy(),data["CDI"]["holo"]["bsmask"]["-"].copy()
retrieved_p_pc,retrieved_n_pc=data["CDI_pc"]["+"].copy(),data["CDI_pc"]["-"].copy()
supportmask,supportmask2=data["CDI"]["supportmask"].copy(),data["CDI"]["supportmask2"].copy()
gamma_p,gamma_n=data["CDI_pc"]["gamma"]["+"].copy(),data["CDI_pc"]["gamma"]["-"].copy()
prop_dist_pc, phase_pc,dx_pc, dy_pc = data["CDI_pc"]["focus"]["prop_dist"].copy(),data["CDI_pc"]["focus"]["phase"].copy(),data["CDI_pc"]["focus"]["dx"].copy(),data["CDI_pc"]["focus"]["dy"].copy()
prop_dist, phase,dx, dy = data["CDI"]["focus"]["prop_dist"].copy(),data["CDI"]["focus"]["phase"].copy(),data["CDI"]["focus"]["dx"].copy(),data["CDI"]["focus"]["dy"].copy()
